# Run weighted average operation

**Rooki** calls climate data operations on the **rook** processing service.

In [ ]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import rooki

**parameters of average operation**

In [ ]:
rooki.weighted_average?

## weighted average

In [ ]:
resp = rooki.weighted_average(
    collection='c3s-cmip6.ScenarioMIP.INM.INM-CM5-0.ssp245.r1i1p1f1.day.tas.gr1.v20190619',
)
resp.ok

In [ ]:
#ds = resp.datasets()[0]
#ds

In [ ]:
from IPython.display import Image
Image(resp.provenance_image())

## workflow: subset + average 


In [ ]:
from rooki import operators as ops

In [ ]:
tas = ops.Input(
    'tas', ['c3s-cmip6.ScenarioMIP.INM.INM-CM5-0.ssp245.r1i1p1f1.day.tas.gr1.v20190619']
)

wf = ops.Subset(tas, time="2016/2016")

wf = ops.WeightedAverage(wf)

In [ ]:
resp = wf.orchestrate()
resp.ok 

In [ ]:
resp.status

In [ ]:
ds = resp.datasets()[0]
ds

In [ ]:
from IPython.display import Image
Image(resp.provenance_image())